In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
print('Number of categories:', dataset.num_classes)

Number of categories: 7


In [4]:
g = dataset[0]

In [5]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [6]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

In [7]:
from dgl.nn.pytorch import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [8]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [9]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.945, val acc: 0.144 (best 0.144), test acc: 0.135 (best 0.135)
In epoch 5, loss: 1.882, val acc: 0.522 (best 0.522), test acc: 0.559 (best 0.559)
In epoch 10, loss: 1.794, val acc: 0.664 (best 0.664), test acc: 0.673 (best 0.673)
In epoch 15, loss: 1.680, val acc: 0.694 (best 0.694), test acc: 0.705 (best 0.705)
In epoch 20, loss: 1.542, val acc: 0.702 (best 0.704), test acc: 0.716 (best 0.714)
In epoch 25, loss: 1.384, val acc: 0.704 (best 0.706), test acc: 0.722 (best 0.718)
In epoch 30, loss: 1.213, val acc: 0.718 (best 0.718), test acc: 0.737 (best 0.737)
In epoch 35, loss: 1.038, val acc: 0.724 (best 0.724), test acc: 0.747 (best 0.747)
In epoch 40, loss: 0.868, val acc: 0.734 (best 0.734), test acc: 0.749 (best 0.749)
In epoch 45, loss: 0.712, val acc: 0.738 (best 0.740), test acc: 0.753 (best 0.754)
In epoch 50, loss: 0.576, val acc: 0.742 (best 0.742), test acc: 0.761 (best 0.761)
In epoch 55, loss: 0.463, val acc: 0.742 (best 0.744), test acc: 0.765 (best 0